In [91]:
## Referrence: 
## https://blog.floydhub.com/colorizing-b&w-photos-with-neural-networks/

from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, grey2rgb
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import scipy.misc  
import os.path  

In [93]:
file_dir = 'train/'
save_path = 'result/'
classes = {'InitImages'}
image_list = []
for index,name in enumerate(classes):
    class_path=file_dir+name+'/'#读取类路径
    for img_name in os.listdir(class_path):
        if img_name != ".DS_Store":
            filenames=class_path+img_name#获取每一个图片地址
            image_list.append(filenames)
print(image_list)
filename_queue = tf.train.string_input_producer(image_list)         
reader = tf.WholeFileReader()          
key,value = reader.read(filename_queue)          
images = tf.image.decode_jpeg(value)#tf.image.decode_png(value)  
CP_H = 256
CP_W = 256
# 裁切图片
with tf.Session() as sess:
    coord = tf.train.Coordinator()  
    threads = tf.train.start_queue_runners(coord=coord)  

    reshapeimg = tf.image.resize_images(images,(CP_H,CP_W))
    for index in range(len(image_list)):
        #reimg1的类型是<class 'numpy.ndarray'>  
        reimg = reshapeimg.eval()  
        scipy.misc.imsave(save_path+'crop_or_pad'+"%03d" %index+'.jpg', reimg)

    coord.request_stop()  
    coord.join(threads)    
print('crop_or_pad successful!')

In [94]:
# Get images
# Change to '/data/images/Train/' to use all the 10k images
X = []
# for filename in os.listdir('../Train/'):
#     X.append(img_to_array(load_img('../Train/'+filename)))
for filename in os.listdir('ImgReshape/crop_or_pad/'):
    if filename != ".DS_Store":
        X.append(img_to_array(load_img('ImgReshape/crop_or_pad/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
# split = int(0.95*len(X))
split = int(len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
Xtrain.shape

In [84]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [95]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 2
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="output/beta_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=1)

In [86]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [96]:
X_ = np.array(img_to_array(load_img('ImgReshape/result/1.png')), dtype=float)
print(X_.shape)
Xtest = rgb2lab(1.0/255*X_)[:,:,0]
Xtest = Xtest.reshape(1, 256, 256, 1)                
                
print(Xtest.shape)
output = model.predict(Xtest)
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = Xtest[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("test_result.jpg", lab2rgb(cur))
imsave("test_gray_version.jpg", rgb2gray(lab2rgb(cur)))
from PIL import Image
image = Image.open('test_result.jpg')
image.show()
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

In [18]:
# Change to '/data/images/Test/' to use all the 500 images
from skimage import io
color_me = []
for filename in os.listdir('../Test/'):
	color_me.append(img_to_array(load_img('../Test/'+filename)))
    
image = io.imread('../Test/0fAtAB.jpg')
print(image.shape)

color_me = np.array(color_me, dtype=float)
print(color_me.shape)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = color_me[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("result/img_"+str(i)+".png", lab2rgb(cur))

(256, 256, 3)
(8, 256, 256, 3)


/Users/jane/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
